In [1]:
%reload_ext Cython

In [2]:
%%cython -f
from libc.stdlib cimport rand, RAND_MAX

cpdef _random():
    return rand() / RAND_MAX

First, we assign a $Preferences$ class which will store 2D preferences matrices $P^A$ and $P^B$. We will use the function $Shape(M)$ to denote the dimensionality of a matrix, such that $Shape(M) \mapsto (m,n)$, where $(m,n)$ represents the dimensionality of an $m \times n$ matrix $M$.

The number of members $A_n \in \mathbb{A}$ and number of members $B_n \in \mathbb{B}$, shall be represented with $a$ and $b$ respectively. Accordingly, the Preferences Matrices of Groups A and B, $P^A$ and $P^B$, will take shapes $a \times b$ and $b \times a$ respectively, such that:

$$
\begin{equation*}
Shape(P^A) \mapsto (a, b); Shape(P^B) \mapsto (b, a).
\label{eq:shape} \tag{1}
\end{equation*}
$$

$P^A_i$ shall refer to $A_i$'s preferences list, such that $P^A_i [n]$ refers to $A_i$'s $n$th preference.

$$
\begin{equation*}
P^A =
\begin{bmatrix}
    \vec{P^A_{1}} \\
    \vec{P^A_{2}} \\
    \vdots \\
    \vec{P^A_a}
\end{bmatrix}
=
\begin{bmatrix}
    P^A_{11} & P^A_{12} & \dots & P^A_{1b} \\
    P^A_{21} & P^A_{22} & \dots & P^A_{2b} \\
    \vdots & \vdots & \ddots & \vdots \\
    P^A_{a1} & P^A_{a2} & \dots & P^A_{ab} \\
\end{bmatrix}; \space
P^B =
\begin{bmatrix}
    \vec{P^B_{1}} \\
    \vec{P^B_{2}} \\
    \vdots \\
    \vec{P^B_b}
\end{bmatrix}
=
\begin{bmatrix}
    P^B_{11} & P^B_{12} & \dots & P^B_{1a} \\
    P^B_{21} & P^B_{22} & \dots & P^B_{2a} \\
    \vdots & \vdots & \ddots & \vdots \\
    P^B_{b1} & P^B_{b2} & \dots & P^B_{ba} \\
\end{bmatrix}.
\label{eq:preferences} \tag{2}
\end{equation*}
$$

Consider an example case where $N_A$ and $N_B$ are both 3, meaning $Shape(P_B)$ and $Shape(P_A)$ will both be $(3, 3)$ square matrices. Preference rankings will be random for this example.

The current matches for $A$ and $B$ will be encoded in matrices denoted as $M^A$ and $M^B$ respectively, where $M^A_{31} = 1$ represents that there is a match between $A_3$ and $B_1$, $M^B_{23} = 0$ represents that there is not a match between $B_2$ and $A_3$, and so on. The initial state of these matrices, $M^A_i$ and $M^B_i$, will be full of zeroes and of shapes $a \times b$ and $b \times a$ respectively.

# NOTE
Changing logic so $P^A_{n,m}$ represent's $A_n$'s preference of $B_m$.

## Schematic

$Preferences$

Stores preference rankings for groups $\mathbb A$ and $\mathbb B$.

```Java
public AbstractSMA.Preferences
```

***

$P^A, P^B$

Contains raw matrix data for $P^A$ and $P^B$ at `Preferences.A` and `Preferences.B`.

```Java
public AbstractSMA.Preferences.A, AbstractSMA.Preferences.B
```

***

$M^A, M^B$

Matrix containing a matrix of binaries representing matches between group members.

```Java
public AbstractSMA.Matches.A, AbstractSMA.Matches.B
```

In [266]:
%%cython -f
# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

cimport cython
from cython.parallel import parallel, prange
from libc.stdlib cimport rand, RAND_MAX

import cupy

from random import random
from cpython cimport array
import numpy as np
cimport numpy as np

ctypedef np.int_t DTYPE_t
cdef bint DEBUG

## DATA GENERATION
@cython.boundscheck(False)
@cython.wraparound(False)
cdef np.ndarray generateData(long i, long j):
    
    cdef long[:,:] output = np.zeros((i,j), dtype=np.int)
    cdef long i_n, j_n
    
    with nogil, parallel():
        for i_n in range(i):
            for j_n in range(j):
                output[i_n,j_n] = j_n + 1
    
    for row in output:
        np.random.shuffle(row)
        
    return np.array(output, dtype=np.int)

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef inline np.ndarray getMaxMask(np.ndarray matrix):
    cdef np.ndarray maxMatrix = np.amax(matrix, axis = 1, keepdims = True)
    return matrix == maxMatrix

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef inline np.ndarray maskWithMax(np.ndarray matrix):
    return matrix * getMaxMask(matrix)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline bint noEmptyRows(np.ndarray matrix):
    cdef int numRows = matrix.shape[0]
    return np.sum(np.amax(matrix, axis = 1, keepdims = True) > 0) >= numRows

        
cdef class AbstractSMA:
    
    cdef public:
        int t, a, b
        np.ndarray PA, PB
        np.ndarray MA, MB
        np.ndarray HA, HB
        
    def __init__(self, np.ndarray PA, np.ndarray PB):
        
        cdef:
            int a = PA.shape[0]
            int b = PB.shape[1]
        
        self.a, self.b = a, b
        self.PA, self.PB = PA, PB

        if(DEBUG): print("\nPA (*)\n", self.PA, "\n\nPB\n", self.PB, "\n")
        
        # initialize MA, MB, HA, HB to np.zeros<int64> of size (a x b)
        self.MA, self.MB = np.zeros(a, dtype=np.int), np.zeros(b, dtype=np.int)
        self.HA = self.HB = np.zeros((a, b), dtype=np.int)
    
    @cython.boundscheck(False)
    @cython.wraparound(False)
    cdef AbstractSMA nextState(self):
        
        cdef:
            # where A is max
            np.ndarray proposerMask = (self.PA == max(0, self.a - self.t))
            # B's rankings of A
            np.ndarray proposerRankings = self.PB * proposerMask
        
        # tracks past proposals
        self.HA = self.HA | proposerMask
        
        # new match is max of prev match and proposer
        self.MB = self.MB | proposerRankings
            
        # start @ t = 0
        if(DEBUG): 
            print("\nt =", self.t, "\n")
            print("Proposer Mask\n", proposerMask, "\n\n")
            print("Proposer Rankings\n", proposerRankings, "\n\n")
            print("History A\n", self.HA, "\n\n")
        
        # only one match per member
        # MA is PA masked by MB
        self.MB = self.MB * getMaxMask(self.MB)
        self.MA = self.PA * (self.MB != 0)
        
        if(DEBUG):
            print("final MA\n", self.MA, "\n\n")
            print("final MB\n", self.MB, "\n\n")

        self.t += 1
        return self
    
    cdef bint everyoneIsContent(self):
#         return np.sum(self.HA) == self.HA.size
        return self.t >= self.a
    
    cdef AbstractSMA run(self):
        while not self.everyoneIsContent():
            self.nextState()
        
        return self

cdef:
#     np.ndarray PA = np.array([
#         [1, 3, 2],
#         [1, 2, 3],
#         [1, 3, 2]
#     ])
    
#     # change to size a x b
#     np.ndarray PB = np.array([
#         [2, 1, 3],
#         [3, 2, 1],
#         [1, 3, 2]
#     ])
    
    int N = 10
    np.ndarray PA = generateData(N, N)
    np.ndarray PB = generateData(N, N)

cpdef test():
    return AbstractSMA(PA, PB).run()

print("Group A is making all proposals for this test case.")

DEBUG = False
myTest = test()
print("Done. Finished in", myTest.t, "steps.")
print("\nMA\n", myTest.MA, "\n\n")
print("MB\n", myTest.MB, "\n\n")

Group A is making all proposals for this test case.
Done. Finished in 10 steps.

MA
 [[5 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 5 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 6 0 0 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 6 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 5 0 0 0 0 0]
 [7 0 0 0 0 0 0 0 0 0]] 


MB
 [[10  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10]
 [ 0  0 10  0  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 10  0]
 [ 0  0  0  0  0  0  0  0 10  0]
 [ 0  0  0  0 10  0  0  0  0  0]
 [10  0  0  0  0  0  0  0  0  0]] 




In [272]:
%timeit -r 1 -n 1 test()

429 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [224]:
PA = np.array([
    [1, 3, 2],
    [1, 2, 3],
    [1, 3, 2]
])
    
PB = np.array([
    [2, 1, 3],
    [3, 2, 1],
    [0, 3, 2]
])

test = np.array([
    [0, 1, 1],
    [0, 1, 0],
    [1, 0, 0]
])

test2 = np.array([
    [1, 0, 1],
    [1, 1, 0],
    [1, 0, 0]
])

test | test2

array([[1, 1, 1],
       [1, 1, 0],
       [1, 0, 0]])

## Expected Output

```
MA (a x b)
 [[0 0 2]
 [1 0 0]
 [0 3 0]] 


MB (a x b)
 [[0 0 3]
 [3 0 0]
 [0 3 0]] 
```

## Todo

- create a utility class that will expose underlying `np.ndarray`s
- store `PA`, `PB`, `MA`, `MB`, etc. in this utility class, ie `RelationshipState`
- add a new state property called `ProposalHistory` that tracks previous proposals

In [144]:
PA = np.array([
    [1, 3, 2, 4],
    [1, 2, 3, 4],
    [1, 3, 2, 4],
    [1, 3, 2, 4]
])

PB_T = np.array([
    [2, 1, 3, 4],
    [3, 2, 1, 4],
    [1, 3, 2, 4],
    [1, 3, 2, 4]
    
])

maskWithMax(PA + PB_T) * 1

array([[0, 0, 0, 8],
       [0, 0, 0, 8],
       [0, 0, 0, 8],
       [0, 0, 0, 8]])

In [70]:
testa = np.random.random((1000,1000))
def runtesta():
    return np.sum(np.amax(testa, axis = 1, keepdims = True) > 0)

testb = np.random.random((1000,1000))
def runtestb():
    return np.sum(np.amax(testb, axis = 1, keepdims = True) > 0)

runtestb()

1000

In [75]:
%timeit -r 10 -n 10 runtesta()

523 µs ± 70.9 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


# Garbage Bin

### Code below deprecated

In [50]:
# MUST 1-INDEX!

PA = np.array([
    [1, 2, 3],
    [1, 3, 2],
    [2, 1, 3]
])

PB = np.array([
    [1, 3, 2],
    [2, 1, 3],
    [2, 1, 0]
])


MA = np.zeros((3,3))
MB = np.zeros((3,3))

out = (PA == 3) * PA
print(out, "\n\n", out == np.amax(out, axis=1))

[[0 0 3]
 [0 3 0]
 [0 0 3]] 

 [[False False  True]
 [False  True False]
 [False False  True]]


In [539]:
%%cython -f --compile-args=-fopenmp --link-args=-fopenmp

import cython
import numpy as np
cimport numpy as np

from cython.parallel import parallel, prange
from libc.stdlib cimport rand, RAND_MAX
cimport cyrandom

# types
from libc.stdint cimport int32_t, uint8_t, uint16_t, uint32_t



cdef class PreferencesData:
    
    cdef DTYPE_t[:,:] matrix
    
    def __init__(self, np.ndarray data):
        self.matrix = data
        
        cdef Py_ssize_t[:] shape = self.matrix.shape
        cdef int i, j
        
        for i in range(shape[0]):
            for j in range(shape[1]):
                self.matrix[i][j] = 0
        
    cpdef np.ndarray array(self):
        return np.array(self.matrix)


ctypedef np.int_t DTYPE_t

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef generateData(int i, int j):
    
    cdef int[:,:] out = np.empty((i, j), dtype=np.int32)
    cdef int i_n, j_n
    cdef int[:] col_idx
    cdef double _rand
    
    with cython.nogil, parallel():
        for i_n in prange(i):
            for j_n in prange(j):
                out[i_n,j_n] = rand() / RAND_MAX > 0.5
#                 np.random.shuffle(out[i_n])
        
    return out

print(np.asarray(generateData(5,5)))

[[1 1 1 0 1]
 [0 1 1 0 0]
 [1 0 0 0 1]
 [0 0 1 1 1]
 [1 0 1 0 0]]


In [559]:
%timeit -r 1000 -n 10 generateData(10,10)

The slowest run took 30.28 times longer than the fastest. This could mean that an intermediate result is being cached.
171 µs ± 169 µs per loop (mean ± std. dev. of 1000 runs, 10 loops each)


In [518]:
%%cython -f 
import cython
import numpy as np
cimport numpy as np
from cython.parallel import parallel, prange

@cython.boundscheck(False)
@cython.wraparound(False)
def matrix_multiply2(double[:,:] u, double[:, :] v, double[:, :] res):
    cdef int i, j, k
    
    cdef int m = u.shape[0]
    cdef int n = u.shape[1]
    cdef int p = v.shape[1]

    with cython.nogil, parallel():
        for i in prange(m):
            for j in prange(p):
                res[i,j] = 0
                for k in range(n):
                    res[i,j] += u[i,k] * v[k,j]

In [266]:
## WORKING NOGIL
# Much faster without fopenmap???
u = np.random.random((10,20))
v = np.random.random((20,5))
res = np.zeros((u.shape[0], v.shape[1]))
%timeit -r10 -n10 matrix_multiply2(u, v, res)

The slowest run took 23.90 times longer than the fastest. This could mean that an intermediate result is being cached.
7.55 µs ± 13.4 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [20]:
cdef class Person:
    def __init__(self):
        pass

cdef class PersonList:
    cdef Person[:] people
    
    def __init__(self, int numPeople):
        self.allocate(numPeople)
        
    cdef void allocate(int numPeople):
        pass


Error compiling Cython file:
------------------------------------------------------------
...
from cpython cimport array

cdef list test = [1,2,3]
cdef int[:] testInts = array.array("i", test)
with nogil:
    for t_i in testInts:
              ^
------------------------------------------------------------

/home/christian/.cache/ipython/cython/_cython_magic_b226d6494c41071575c828ace054c84b.pyx:6:15: Iterating over Python object not allowed without gil

Error compiling Cython file:
------------------------------------------------------------
...
from cpython cimport array

cdef list test = [1,2,3]
cdef int[:] testInts = array.array("i", test)
with nogil:
    for t_i in testInts:
              ^
------------------------------------------------------------

/home/christian/.cache/ipython/cython/_cython_magic_b226d6494c41071575c828ace054c84b.pyx:6:15: Converting to Python object not allowed without gil

Error compiling Cython file:
---------------------------------------------------------

In [127]:
%%cython -f

import numpy as np
cimport numpy as np
cimport cython # so we can use cython decorators
from cpython cimport bool # type annotation for boolean

# disable index bounds checking and negative indexing for speedups
@cython.wraparound(False)
@cython.boundscheck(False)
cpdef cython_get_sample(np.ndarray arr, 
                        arr_len,
                        n_iter, 
                        int sample_size, 
                        bool fast = False):
    
    cdef int start_idx
    if fast:
        start_idx = (n_iter * sample_size) % arr_len
        if start_idx + sample_size >= arr_len:
            np.random.shuffle(arr)
            
        return arr[start_idx:start_idx+sample_size] 
    else:
        return np.random.choice(arr, sample_size, replace=False)